In [ ]:
pip install horovod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 61.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for horovod: filename=horovod-0.28.1-cp310-cp310-linux_x86_64.whl size=41445854 sha256=de55478aec8c5175b96b3fa19cb0d19078dfce48ebb15281ee7081589f2b43c4
  Stored in directory: /root/.cache/pip/wheels/68/48/5f/ebbf9b83e04c68070cf9621e6466c33d698efb747d7920b81f
Successfully built horovod


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
import os
from keras.optimizers import Adam
import tensorflow as tf
import horovod.keras as hvd
import time

In [ ]:
hvd.init()

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.visible_device_list = str(hvd.local_rank())
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))


In [ ]:
metadata = pd.read_csv('/content/drive/MyDrive/data/original.csv')

In [ ]:
metadata.head(10)


,participant,filename,sound_prediction_score,tb_status
0,CODA_TB_0001,1645088710003-recording-1.wav,0.990254,0
1,CODA_TB_0001,1645088760390-recording-1.wav,0.990272,0
2,CODA_TB_0001,1645088760830-recording-1.wav,0.990112,0
3,CODA_TB_0001,1645088710843-recording-1.wav,0.990152,0
4,CODA_TB_0001,1645088759950-recording-1.wav,0.990039,0
5,CODA_TB_0001,1645088758630-recording-1.wav,0.990004,0
6,CODA_TB_0001,1645088758230-recording-1.wav,0.975063,0
7,CODA_TB_0001,1645088738844-recording-1.wav,0.990068,0
8,CODA_TB_0001,1645088784942-recording-1.wav,0.990291,0
9,CODA_TB_0001,1645088784542-recording-1.wav,0.990288,0


In [ ]:
metadata['tb_status'].value_counts()

0    13684
1    13684
Name: tb_status, dtype: int64

In [ ]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27368 entries, 0 to 27367
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   participant             27368 non-null  object 
 1   filename                27368 non-null  object 
 2   sound_prediction_score  27368 non-null  float64
 3   tb_status               27368 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 855.4+ KB


In [ ]:
# audio_dataset_path = 'time/'
# audio_dataset_path = '/content/drive/MyDrive/data/time'

In [ ]:
import json

# Specify the path to the JSON file
input_file_path = '/content/drive/MyDrive/data/extracted_features.json'

# Read the JSON file
with open(input_file_path, 'r') as json_file:
    data = json.load(json_file)

# Extract the 'extracted_features' list from the loaded data
extracted_features = data['extracted_features']

In [ ]:
extracted_features_df = pd.DataFrame(extracted_features, columns = ['feature', 'tb_status'])
extracted_features_df.head()

,feature,tb_status
0,"[-233.81036376953125, 60.20777893066406, 13.18...",0
1,"[-227.28358459472656, 66.78350067138672, 26.54...",0
2,"[-291.9474182128906, 61.16533279418945, 36.864...",0
3,"[-322.38006591796875, 53.53618240356445, 30.72...",0
4,"[-215.52008056640625, 58.504398345947266, 26.0...",0


In [ ]:
X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['tb_status'].tolist())

In [ ]:
X.shape


(27368, 40)

In [ ]:
y.shape

(27368,)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = to_categorical(y, num_classes = 2)

In [ ]:
y.shape

(27368, 2)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
print(X_train.shape, " ", X_test.shape)
print(y_train.shape, " ", y_test.shape)

(21894, 40)   (5474, 40)
(21894, 2)   (5474, 2)


In [ ]:
X_train = X_train.reshape(X_train.shape[0], 8, 5, 1)
X_test = X_test.reshape(X_test.shape[0], 8, 5, 1)
print(X_train.shape, " ", X_test.shape)

(21894, 8, 5, 1)   (5474, 8, 5, 1)


In [ ]:
input_dim = (8, 5, 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
model = Sequential()

model.add(Conv2D(32, (2, 2), activation='relu', input_shape = input_dim))
model.add(MaxPooling2D((1, 1)))

model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 1)))

model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 1)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 4, 32)          160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 4, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 3, 64)          8256      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 3, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 2, 128)         32896     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 2, 128)        0

In [ ]:
optimizer = tf.keras.optimizers.Adam()

# Wrap the optimizer with Horovod
optimizer = hvd.DistributedOptimizer(optimizer)

In [ ]:
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
callbacks = [
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
    hvd.callbacks.MetricAverageCallback(),
    hvd.callbacks.LearningRateWarmupCallback(
        warmup_epochs=5,
        initial_lr=0.001,
        verbose=1
    ),
]




In [ ]:
start_time = time.time()

model.fit(
    X_train, y_train,
    batch_size=32,
    callbacks=callbacks,
    epochs=20,
    verbose=1 if hvd.rank() == 0 else 0
)
end_time = time.time()
total_duration = end_time - start_time

Epoch 1/20
685/685 [==============================] - 4s 6ms/step - loss: 0.0143 - accuracy: 0.9951 - lr: 0.0010
Epoch 2/20
685/685 [==============================] - 4s 5ms/step - loss: 0.0302 - accuracy: 0.9916 - lr: 0.0010
Epoch 3/20
685/685 [==============================] - 4s 6ms/step - loss: 0.0224 - accuracy: 0.9932 - lr: 0.0010
Epoch 4/20
685/685 [==============================] - 4s 6ms/step - loss: 0.0195 - accuracy: 0.9943 - lr: 0.0010
Epoch 5/20
678/685 [============================>.] - ETA: 0s - loss: 0.0338 - accuracy: 0.9902
Epoch 5: finished gradual learning rate warmup to 0.001.
685/685 [==============================] - 4s 5ms/step - loss: 0.0338 - accuracy: 0.9902 - lr: 0.0010
Epoch 6/20
685/685 [==============================] - 3s 4ms/step - loss: 0.0178 - accuracy: 0.9943 - lr: 0.0010
Epoch 7/20
685/685 [==============================] - 4s 5ms/step - loss: 0.0209 - accuracy: 0.9945 - lr: 0.0010
Epoch 8/20
685/685 [==============================] - 4s 5ms/step -

In [ ]:
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_acc:.4f}')

print(f'Total Training Duration using GPU : {total_duration:.2f} seconds')

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')



Train Loss: 0.0016
Train Accuracy: 0.9996
Total Training Duration using GPU : 82.04 seconds
Test Loss: 0.2086
Test Accuracy: 0.9682
